In [9]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time

def crawl_item(url_input):
  try:
    url = 'https://mediamart.vn' + url_input
    headers = {
      'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:73.0) Gecko/20100101 Firefox/73.0',
      'X-Requested-With': 'XMLHttpRequest',
    }    
    r = requests.get(url, headers=headers)
    data = {}
    soup = BeautifulSoup(r.content.decode('utf-8', 'ignore'), 'html.parser')
    data["name"] = soup.find_all("div", class_="pdetail-name")[0].text.split('\n')[1]
    x=0
    key=''
    for tr in soup.find_all('td'):
      if(len(tr.get_text()) > 0):
        if(x%2)==0:
          key=tr.get_text()
        else:
          data[key]=tr.get_text()
        x=x+1
    data["price"] = soup.find("div", class_="pdetail-price-box").text
    time.sleep(10)
    return data
  except:
    pass

def crawl_mediaMart():
  data=[]
  df = pd.read_csv('mediamart.csv') #duong dan file csv
  url = 'https://mediamart.vn/smartphones'
  headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:73.0) Gecko/20100101 Firefox/73.0',
    'X-Requested-With': 'XMLHttpRequest',
  }    
  r = requests.get(url, headers=headers)
  soup = BeautifulSoup(r.content, 'html.parser')
  for div in soup.find_all("div", class_="col-6 col-md-3 col-lg-3"):
    data.append(crawl_item(div.find("a", href=True)['href']))
  pd.concat([pd.DataFrame(data), df], ignore_index=True)
  return df

In [10]:
!pip install beautifulsoup4

You should consider upgrading via the 'C:\Users\Admin\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [ ]:
url = 'https://mediamart.vn/smartphones/apple-iphone-11-64gb-white'
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:73.0) Gecko/20100101 Firefox/73.0',
    'X-Requested-With': 'XMLHttpRequest',
}    
r = requests.get(url, headers=headers)
data = {}
soup = BeautifulSoup(r.content.decode('utf-8', 'ignore'), 'html.parser')

data["name"] = soup.find_all("div", class_="pdetail-name")[0].text.split('\n')[1]
x=0
key=''
for tr in soup.find_all('td'):
  if(len(tr.get_text()) > 0):
    if(x%2)==0:
      key=tr.get_text()
    else:
      data[key]=tr.get_text()
    x=x+1
data["price"] = soup.find("div", class_="pdetail-price-box").text
print(data)
jsonString = json.dumps(data, ensure_ascii=False)
jsonFile = open("/content/drive/MyDrive/Colab Notebooks/Media_mart/1.json", "w")
jsonFile.write(jsonString)
jsonFile.close()

{'name': 'Apple Iphone 11 64G White', 'Thương hiệu': '\nApple\n', 'Mã sản phẩm': '\nIphone 11\n', 'Hộp máy gồm': 'Sách hướng dẫnCáp Lightning - Type CCây lấy sim', 'Công nghệ màn hình': 'Retina', 'Độ phân giải': '1792 x 828 pixels', 'Màn hình': '6.1 inch', 'Cảm ứng': 'Đa điểm', 'Mặt kính cảm ứng': 'Kính cường lực', 'Camera sau': 'Kép 12MP + 12MP', 'Quay phim': 'Có', 'Đèn Flash': 'Có', 'Chụp ảnh nâng cao': 'Chụp chế độ ban đêmChạm lấy nétChế độ ánh sáng yếuTự động lấy nét', 'Camera trước': '12 MP', 'Videocall': 'Có', 'Thông tin khác': 'Tự động lấy nétNhận diện khuôn mặt', 'Hệ điều hành': 'iOS13', 'CPU': 'A13 Bionic', 'Số nhân': '6 nhân', 'Tốc độ tối đa': '2.66 GHz', 'RAM': '4 GB', 'Bộ nhớ': '64 GB', 'Băng tần 2G': 'Có', 'Băng tần 3G': 'Có', 'Hỗ trợ 4G': 'Có hỗ trợ 4G', 'Số khe sim': '1 Nano SIM & 1 eSIM', 'Wifi': 'Có', 'GPS': 'Có', 'Bluetooth': 'Có', 'NFC': 'Có', 'Cổng kết nối/sạc': 'Lightning', 'Kết nối khác': 'NFC', 'Thiết kế': 'Nguyên khối', 'Chất liệu': 'Khung nhôm, mặt lưng k

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
f = open("./Media_Mart.txt", "r")
soup = BeautifulSoup(f.read(), 'html.parser')

In [ ]:
soup

In [ ]:
images_link = []
for img in soup.find_all("div", class_ = "card-img-top"):
  images_link.append(img.find("img")['src'])

In [ ]:
images_link = images_link[:147]

In [ ]:
len(soup.find_all("div", class_="col-6 col-md-3 col-lg-3"))

147

In [ ]:
import time
k = 0
with open("Media_Mart.txt", mode = "w", encoding = "utf8") as f:
  for div in soup.find_all("div", class_="col-6 col-md-3 col-lg-3"):
    try:
      url = 'https://mediamart.vn' + div.find("a", href=True)['href']
      headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:73.0) Gecko/20100101 Firefox/73.0',
        'X-Requested-With': 'XMLHttpRequest',
      }    
      r = requests.get(url, headers=headers)
      data = {}
      detail = BeautifulSoup(r.content.decode('utf-8', 'ignore'), 'html.parser')
      data["name"] = detail.find_all("div", class_="pdetail-name")[0].text.split('\n')[1]
      data["url"] = url
      data["image_link"] = images_link[k]
      x=0
      key=''
      for tr in detail.find_all('td'):
        if(len(tr.get_text()) > 0):
          if(x%2)==0:
            key=tr.get_text()
          else:
            data[key]=tr.get_text()
          x=x+1
      data["price"] = detail.find("div", class_="pdetail-price-box").text
      jsonString = json.dumps(data, ensure_ascii=False)
      f.write(jsonString + "\n")
      print(jsonString)
      k = k + 1
    except:
      pass
    # time.sleep(2)


In [ ]:
import json
import pandas as pd

data_to_csv = []
with open("Media_Mart.txt", mode = "r", encoding="utf8") as f:
  for idx, line in enumerate(f):
    data = json.loads(line)
    data_to_csv.append(data)
print(data_to_csv)

In [57]:
for x in data_to_csv:
    print(x)
    print(x.keys())
    break

{'name': 'Apple Iphone 11 64G White', 'url': 'https://mediamart.vn/smartphones/apple-iphone-11-64gb-white', 'image_link': 'https://cdn.mediamart.vn/thumb/images/product/-N9QObu.jpg', 'Thương hiệu': '\nApple\n', 'Mã sản phẩm': '\nIphone 11\n', 'Hộp máy gồm': 'Sách hướng dẫnCáp Lightning - Type CCây lấy sim', 'Công nghệ màn hình': 'Retina', 'Độ phân giải': '1792 x 828 pixels', 'Màn hình': '6.1 inch', 'Cảm ứng': 'Đa điểm', 'Mặt kính cảm ứng': 'Kính cường lực', 'Camera sau': 'Kép 12MP + 12MP', 'Quay phim': 'Có', 'Đèn Flash': 'Có', 'Chụp ảnh nâng cao': 'Chụp chế độ ban đêmChạm lấy nétChế độ ánh sáng yếuTự động lấy nét', 'Camera trước': '12 MP', 'Videocall': 'Có', 'Thông tin khác': 'Tự động lấy nétNhận diện khuôn mặt', 'Hệ điều hành': 'iOS13', 'CPU': 'A13 Bionic', 'Số nhân': '6 nhân', 'Tốc độ tối đa': '2.66 GHz', 'RAM': '4 GB', 'Bộ nhớ': '64 GB', 'Băng tần 2G': 'Có', 'Băng tần 3G': 'Có', 'Hỗ trợ 4G': 'Có hỗ trợ 4G', 'Số khe sim': '1 Nano SIM & 1 eSIM', 'Wifi': 'Có', 'GPS': 'Có', 'Bluetooth

In [96]:
data_new = []
for pair in data_to_csv:
    dt = {}
    for key, value in pair.items():
        key = str(key).strip().replace('\n','')
        dt[key] = value
    data_new.append(dt)

In [97]:
pd.DataFrame(data_new).to_csv('mediamart.csv')

In [98]:
df = pd.read_csv("mediamart.csv", index_col=0)

In [101]:
df["Thương hiệu"] = df["Thương hiệu"].apply(lambda line : str(line).strip())
df["Mã sản phẩm"] = df["Mã sản phẩm"].apply(lambda line : str(line).strip())
df["price"] = df["price"].apply(lambda line : str(line).strip()).apply(lambda line: int(line.split("\n")[0].strip()[:-2].replace(".","")))

In [102]:
df.head()

,name,url,image_link,Thương hiệu,Mã sản phẩm,Hộp máy gồm,Công nghệ màn hình,Độ phân giải,Màn hình,Cảm ứng,...,Mali-T720MP2,1 GB,8 GB,32 GB,1 Nano + 1 Micro,Có,CóA-GPS,Micro USB,2050 mAh,Pin chuẩn Li-Ion
0,Apple Iphone 11 64G White,https://mediamart.vn/smartphones/apple-iphone-...,https://cdn.mediamart.vn/thumb/images/product/...,Apple,Iphone 11,Sách hướng dẫnCáp Lightning - Type CCây lấy sim,Retina,1792 x 828 pixels,6.1 inch,Đa điểm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Điện thoại Samsung Galaxy A13 (4G+128G) A135F Đen,https://mediamart.vn/smartphones/dien-thoai-ga...,https://cdn.mediamart.vn/thumb/images/product/...,Samsung,A13,NaN,TFT LCD,Full HD+ (1080x2408 pixels),6.6 inch,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Apple iphone 11 128G Black,https://mediamart.vn/smartphones/apple-iphone-...,https://cdn.mediamart.vn/thumb/images/product/...,Apple,iphone 11,Sách hướng dẫnCáp Lightning - Type CCây lấy sim,Retina,1792 x 828 pixels,6.1 inch,Đa điểm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Điện thoại OPPO CPH2325 - A55 Đen Mạnh Mẽ,https://mediamart.vn/smartphones/dien-thoai-op...,https://cdn.mediamart.vn/thumb/images/product/...,Oppo,A55,NaN,IPS LCD,700 x 1600 Pixel,6.51 inch,Đa điểm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Điện thoại Xiaomi REDMI 9A (2+32) Xanh lá,https://mediamart.vn/smartphones/dien-thoai-xi...,https://cdn.mediamart.vn/thumb/images/product/...,Xiaomi,Redmi 9A,Sách hướng dẫnCáp Micro USBCây lấy simCủ sạc n...,IPS LCD,700 x 1600 Pixel,6.53 inch,Đa điểm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
for column, value in df.isnull().sum().sort_values(ascending=True).items():
    print(column, value)

name 0
url 0
image_link 0
Thương hiệu 0
Mã sản phẩm 0
price 0
RAM 2
GPS 3
Màn hình 3
Bộ nhớ 4
Số khe sim 5
Công nghệ màn hình 5
Camera trước 5
Độ phân giải 5
Bảo hành 5
Hệ điều hành 5
Wifi 6
Bluetooth 6
Số nhân 7
Thiết kế 7
Camera sau 7
Cổng kết nối/sạc 9
Chụp ảnh nâng cao 11
CPU 11
Quay phim 16
Tính năng đặc biệt 22
Jack tai nghe 23
Thông tin khác 24
Mặt kính cảm ứng 24
Videocall 26
Dung lượng pin 27
Nền tảng 27
Đèn Flash 29
Kết nối khác 29
Chức năng khác 29
Loại pin 30
Chip đồ họa (GPU) 33
Chất liệu 40
Tốc độ tối đa 60
Hộp máy gồm 65
Hỗ trợ 4G 73
Thẻ nhớ ngoài 82
Hỗ Trợ 5G 88
Xuất xứ 90
Tần số quét 94
Ghi âm 96
Hỗ trợ thẻ tối đa 99
Loại Sim 105
Chống nước 106
Cảm ứng 108
Nghe nhạc 112
Xem phim 112
Băng tần 3G 120
Radio 122
Băng tần 2G 123
NFC 134
2050mAh 144
5.0 MP 144
Li-ion 144
Itel 144
Itel A32F 144
CápTai ngheSạcỐp lưng 144
5.0 inch 144
Cảm ứng điện dung 144
Android 8.1 Oreo 144
1 GB 144
1.3 GHz 144
Mali-T720MP2 144
PIN 144
8 GB 144
32 GB 144
1 Nano + 1 Micro 144
Có 144
CóA

In [106]:
df = df.loc[:, df.isnull().sum() < 144]

In [108]:
df.isnull().sum().sort_values(ascending=True)

name                      0
url                       0
image_link                0
Thương hiệu               0
Mã sản phẩm               0
price                     0
RAM                       2
GPS                       3
Màn hình                  3
Bộ nhớ                    4
Bảo hành                  5
Hệ điều hành              5
Camera trước              5
Số khe sim                5
Công nghệ màn hình        5
Độ phân giải              5
Bluetooth                 6
Wifi                      6
Thiết kế                  7
Camera sau                7
Số nhân                   7
Cổng kết nối/sạc      9
Chụp ảnh nâng cao        11
CPU                      11
Quay phim                16
Tính năng đặc biệt       22
Jack tai nghe            23
Mặt kính cảm ứng         24
Thông tin khác           24
Videocall                26
Nền tảng                 27
Dung lượng pin           27
Chức năng khác           29
Kết nối khác             29
Đèn Flash                29
Loại pin            

In [110]:
df.to_csv("mediamart.csv", index=0)

In [111]:
df_new = pd.read_csv("mediamart.csv")

In [113]:
df_new["Màn hình"]

0       6.1 inch
1       6.6 inch
2       6.1 inch
3      6.51 inch
4      6.53 inch
         ...    
142     6.1 inch
143     6.1 inch
144     5.4 inch
145     5.4 inch
146     6.7 inch
Name: Màn hình, Length: 147, dtype: object